<a href="https://colab.research.google.com/github/DariusTheGeek/Radiant-Earth-Spot-the-Crop-Challenge/blob/main/Brainiac_Pixel_Features_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Oct  8 11:04:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip -qq install catboost pytorch_tabular[all]

     |████████████████████████████████| 76.4 MB 38 kB/s 
     |████████████████████████████████| 2.0 MB 76.1 MB/s 
     |████████████████████████████████| 80 kB 9.9 MB/s 
     |████████████████████████████████| 23.1 MB 1.3 MB/s 
     |████████████████████████████████| 809 kB 63.9 MB/s 
     |████████████████████████████████| 74 kB 4.2 MB/s 
     |████████████████████████████████| 282 kB 88.6 MB/s 
     |████████████████████████████████| 595 kB 89.7 MB/s 
     |████████████████████████████████| 1.8 MB 56.3 MB/s 
     |████████████████████████████████| 13.2 MB 58.6 MB/s 
     |████████████████████████████████| 636 kB 66.2 MB/s 
     |████████████████████████████████| 125 kB 93.0 MB/s 
     |████████████████████████████████| 829 kB 61.4 MB/s 
     |████████████████████████████████| 180 kB 77.1 MB/s 
     |████████████████████████████████| 139 kB 89.4 MB/s 
     |████████████████████████████████| 76 kB 6.2 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████

In [3]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from sklearn.model_selection import  StratifiedKFold
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import torch
import random
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Set seed
def setup_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)  # cpu
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = False 
    torch.backends.cudnn.benchmark = True 

seed = 2021
setup_seed(seed) 

In [5]:
# Month Start End Data
!gdown --id 1-3eSgP9jRK3n5JATtd62h0LNnQwOluqv
!gdown --id 1-6d5EMxlg44amERHerYSuVkXCiUFu-kM

# Season Data
!gdown --id 1-955Isi3nJAUd0v7RWohj9zPFhsM4Rvi
!gdown --id 1-7of-3JVM1PII3JMHM7kQQE4ebjGShDm

Downloading...
From: https://drive.google.com/uc?id=1-3eSgP9jRK3n5JATtd62h0LNnQwOluqv
To: /content/month_start_end_train.csv
100% 289M/289M [00:01<00:00, 238MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-6d5EMxlg44amERHerYSuVkXCiUFu-kM
To: /content/month_start_end_test.csv
100% 118M/118M [00:00<00:00, 209MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-955Isi3nJAUd0v7RWohj9zPFhsM4Rvi
To: /content/season_train.csv
100% 458M/458M [00:01<00:00, 274MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-7of-3JVM1PII3JMHM7kQQE4ebjGShDm
To: /content/season_test.csv
100% 186M/186M [00:00<00:00, 228MB/s]


In [6]:
# Load data
month_start_end_train = pd.read_csv('month_start_end_train.csv').sort_values(by = 'field').reset_index(drop = True)
month_start_end_test = pd.read_csv('month_start_end_test.csv').sort_values(by = 'field').reset_index(drop = True)
season_train = pd.read_csv('season_train.csv').sort_values(by = 'field').reset_index(drop = True)
season_test = pd.read_csv('season_test.csv').sort_values(by = 'field').reset_index(drop = True)
season_train.head()

,B01_PERIOD_0,B02_PERIOD_0,B03_PERIOD_0,B04_PERIOD_0,B05_PERIOD_0,B06_PERIOD_0,B07_PERIOD_0,B08_PERIOD_0,B8A_PERIOD_0,B09_PERIOD_0,B11_PERIOD_0,B12_PERIOD_0,CLM_PERIOD_0,B01_PERIOD_1,B02_PERIOD_1,B03_PERIOD_1,B04_PERIOD_1,B05_PERIOD_1,B06_PERIOD_1,B07_PERIOD_1,B08_PERIOD_1,B8A_PERIOD_1,B09_PERIOD_1,B11_PERIOD_1,B12_PERIOD_1,CLM_PERIOD_1,B01_PERIOD_2,B02_PERIOD_2,B03_PERIOD_2,B04_PERIOD_2,B05_PERIOD_2,B06_PERIOD_2,B07_PERIOD_2,B08_PERIOD_2,B8A_PERIOD_2,B09_PERIOD_2,B11_PERIOD_2,B12_PERIOD_2,CLM_PERIOD_2,B01_PERIOD_3,B02_PERIOD_3,B03_PERIOD_3,B04_PERIOD_3,B05_PERIOD_3,B06_PERIOD_3,B07_PERIOD_3,B08_PERIOD_3,B8A_PERIOD_3,B09_PERIOD_3,B11_PERIOD_3,B12_PERIOD_3,CLM_PERIOD_3,B01_PERIOD_4,B02_PERIOD_4,B03_PERIOD_4,B04_PERIOD_4,B05_PERIOD_4,B06_PERIOD_4,B07_PERIOD_4,B08_PERIOD_4,B8A_PERIOD_4,B09_PERIOD_4,B11_PERIOD_4,B12_PERIOD_4,CLM_PERIOD_4,B01_PERIOD_5,B02_PERIOD_5,B03_PERIOD_5,B04_PERIOD_5,B05_PERIOD_5,B06_PERIOD_5,B07_PERIOD_5,B08_PERIOD_5,B8A_PERIOD_5,B09_PERIOD_5,B11_PERIOD_5,B12_PERIOD_5,CLM_PERIOD_5,B01_PERIOD_6,B02_PERIOD_6,B03_PERIOD_6,B04_PERIOD_6,B05_PERIOD_6,B06_PERIOD_6,B07_PERIOD_6,B08_PERIOD_6,B8A_PERIOD_6,B09_PERIOD_6,B11_PERIOD_6,B12_PERIOD_6,CLM_PERIOD_6,B01_PERIOD_7,B02_PERIOD_7,B03_PERIOD_7,B04_PERIOD_7,B05_PERIOD_7,B06_PERIOD_7,B07_PERIOD_7,B08_PERIOD_7,B8A_PERIOD_7,B09_PERIOD_7,B11_PERIOD_7,B12_PERIOD_7,CLM_PERIOD_7,B01_PERIOD_8,B02_PERIOD_8,B03_PERIOD_8,B04_PERIOD_8,B05_PERIOD_8,B06_PERIOD_8,B07_PERIOD_8,B08_PERIOD_8,B8A_PERIOD_8,B09_PERIOD_8,B11_PERIOD_8,B12_PERIOD_8,CLM_PERIOD_8,B01_PERIOD_9,B02_PERIOD_9,B03_PERIOD_9,B04_PERIOD_9,B05_PERIOD_9,B06_PERIOD_9,B07_PERIOD_9,B08_PERIOD_9,B8A_PERIOD_9,B09_PERIOD_9,B11_PERIOD_9,B12_PERIOD_9,CLM_PERIOD_9,B01_PERIOD_10,B02_PERIOD_10,B03_PERIOD_10,B04_PERIOD_10,B05_PERIOD_10,B06_PERIOD_10,B07_PERIOD_10,B08_PERIOD_10,B8A_PERIOD_10,B09_PERIOD_10,B11_PERIOD_10,B12_PERIOD_10,CLM_PERIOD_10,B01_PERIOD_11,B02_PERIOD_11,B03_PERIOD_11,B04_PERIOD_11,B05_PERIOD_11,B06_PERIOD_11,B07_PERIOD_11,B08_PERIOD_11,B8A_PERIOD_11,B09_PERIOD_11,B11_PERIOD_11,B12_PERIOD_11,CLM_PERIOD_11,B01_PERIOD_12,B02_PERIOD_12,B03_PERIOD_12,B04_PERIOD_12,B05_PERIOD_12,B06_PERIOD_12,B07_PERIOD_12,B08_PERIOD_12,B8A_PERIOD_12,B09_PERIOD_12,B11_PERIOD_12,B12_PERIOD_12,CLM_PERIOD_12,B01_PERIOD_13,B02_PERIOD_13,B03_PERIOD_13,B04_PERIOD_13,B05_PERIOD_13,B06_PERIOD_13,B07_PERIOD_13,B08_PERIOD_13,B8A_PERIOD_13,B09_PERIOD_13,B11_PERIOD_13,B12_PERIOD_13,CLM_PERIOD_13,B01_PERIOD_14,B02_PERIOD_14,B03_PERIOD_14,B04_PERIOD_14,B05_PERIOD_14,B06_PERIOD_14,B07_PERIOD_14,B08_PERIOD_14,B8A_PERIOD_14,B09_PERIOD_14,B11_PERIOD_14,B12_PERIOD_14,CLM_PERIOD_14,B01_PERIOD_15,B02_PERIOD_15,B03_PERIOD_15,B04_PERIOD_15,B05_PERIOD_15,B06_PERIOD_15,B07_PERIOD_15,B08_PERIOD_15,B8A_PERIOD_15,B09_PERIOD_15,B11_PERIOD_15,B12_PERIOD_15,CLM_PERIOD_15,B01_PERIOD_16,B02_PERIOD_16,B03_PERIOD_16,B04_PERIOD_16,B05_PERIOD_16,B06_PERIOD_16,B07_PERIOD_16,B08_PERIOD_16,B8A_PERIOD_16,B09_PERIOD_16,B11_PERIOD_16,B12_PERIOD_16,CLM_PERIOD_16,B01_PERIOD_17,B02_PERIOD_17,B03_PERIOD_17,B04_PERIOD_17,B05_PERIOD_17,B06_PERIOD_17,B07_PERIOD_17,B08_PERIOD_17,B8A_PERIOD_17,B09_PERIOD_17,B11_PERIOD_17,B12_PERIOD_17,CLM_PERIOD_17,B01_PERIOD_18,B02_PERIOD_18,B03_PERIOD_18,B04_PERIOD_18,B05_PERIOD_18,B06_PERIOD_18,B07_PERIOD_18,B08_PERIOD_18,B8A_PERIOD_18,B09_PERIOD_18,B11_PERIOD_18,B12_PERIOD_18,CLM_PERIOD_18,B01_PERIOD_19,B02_PERIOD_19,B03_PERIOD_19,B04_PERIOD_19,B05_PERIOD_19,B06_PERIOD_19,B07_PERIOD_19,B08_PERIOD_19,B8A_PERIOD_19,B09_PERIOD_19,B11_PERIOD_19,B12_PERIOD_19,CLM_PERIOD_19,B01_PERIOD_20,B02_PERIOD_20,B03_PERIOD_20,B04_PERIOD_20,B05_PERIOD_20,B06_PERIOD_20,B07_PERIOD_20,B08_PERIOD_20,B8A_PERIOD_20,B09_PERIOD_20,B11_PERIOD_20,B12_PERIOD_20,CLM_PERIOD_20,B01_PERIOD_21,B02_PERIOD_21,B03_PERIOD_21,B04_PERIOD_21,B05_PERIOD_21,B06_PERIOD_21,B07_PERIOD_21,B08_PERIOD_21,B8A_PERIOD_21,B09_PERIOD_21,B11_PERIOD_21,B12_PERIOD_21,CLM_PERIOD_21,B01_PERIOD_22,B02_PERIOD_22,B03_PERIOD_22,B04_PERIOD_22,B05_PERIOD_22,B06_PERIOD_22,B07_PERIOD_22,B08_PERIOD_22,B8A_PERIOD_22

In [7]:
month_start_end_train.head()

,B01_APR_START,B02_APR_START,B03_APR_START,B04_APR_START,B05_APR_START,B06_APR_START,B07_APR_START,B08_APR_START,B8A_APR_START,B09_APR_START,B11_APR_START,B12_APR_START,CLM_APR_START,B01_APR_END,B02_APR_END,B03_APR_END,B04_APR_END,B05_APR_END,B06_APR_END,B07_APR_END,B08_APR_END,B8A_APR_END,B09_APR_END,B11_APR_END,B12_APR_END,CLM_APR_END,B01_MAY_START,B02_MAY_START,B03_MAY_START,B04_MAY_START,B05_MAY_START,B06_MAY_START,B07_MAY_START,B08_MAY_START,B8A_MAY_START,B09_MAY_START,B11_MAY_START,B12_MAY_START,CLM_MAY_START,B01_MAY_END,B02_MAY_END,B03_MAY_END,B04_MAY_END,B05_MAY_END,B06_MAY_END,B07_MAY_END,B08_MAY_END,B8A_MAY_END,B09_MAY_END,B11_MAY_END,B12_MAY_END,CLM_MAY_END,B01_JUN_START,B02_JUN_START,B03_JUN_START,B04_JUN_START,B05_JUN_START,B06_JUN_START,B07_JUN_START,B08_JUN_START,B8A_JUN_START,B09_JUN_START,B11_JUN_START,B12_JUN_START,CLM_JUN_START,B01_JUN_END,B02_JUN_END,B03_JUN_END,B04_JUN_END,B05_JUN_END,B06_JUN_END,B07_JUN_END,B08_JUN_END,B8A_JUN_END,B09_JUN_END,B11_JUN_END,B12_JUN_END,CLM_JUN_END,B01_JUL_START,B02_JUL_START,B03_JUL_START,B04_JUL_START,B05_JUL_START,B06_JUL_START,B07_JUL_START,B08_JUL_START,B8A_JUL_START,B09_JUL_START,B11_JUL_START,B12_JUL_START,CLM_JUL_START,B01_JUL_END,B02_JUL_END,B03_JUL_END,B04_JUL_END,B05_JUL_END,B06_JUL_END,B07_JUL_END,B08_JUL_END,B8A_JUL_END,B09_JUL_END,B11_JUL_END,B12_JUL_END,CLM_JUL_END,B01_AUG_START,B02_AUG_START,B03_AUG_START,B04_AUG_START,B05_AUG_START,B06_AUG_START,B07_AUG_START,B08_AUG_START,B8A_AUG_START,B09_AUG_START,B11_AUG_START,B12_AUG_START,CLM_AUG_START,B01_AUG_END,B02_AUG_END,B03_AUG_END,B04_AUG_END,B05_AUG_END,B06_AUG_END,B07_AUG_END,B08_AUG_END,B8A_AUG_END,B09_AUG_END,B11_AUG_END,B12_AUG_END,CLM_AUG_END,B01_SEPT_START,B02_SEPT_START,B03_SEPT_START,B04_SEPT_START,B05_SEPT_START,B06_SEPT_START,B07_SEPT_START,B08_SEPT_START,B8A_SEPT_START,B09_SEPT_START,B11_SEPT_START,B12_SEPT_START,CLM_SEPT_START,B01_SEPT_END,B02_SEPT_END,B03_SEPT_END,B04_SEPT_END,B05_SEPT_END,B06_SEPT_END,B07_SEPT_END,B08_SEPT_END,B8A_SEPT_END,B09_SEPT_END,B11_SEPT_END,B12_SEPT_END,CLM_SEPT_END,B01_OCT_START,B02_OCT_START,B03_OCT_START,B04_OCT_START,B05_OCT_START,B06_OCT_START,B07_OCT_START,B08_OCT_START,B8A_OCT_START,B09_OCT_START,B11_OCT_START,B12_OCT_START,CLM_OCT_START,B01_OCT_END,B02_OCT_END,B03_OCT_END,B04_OCT_END,B05_OCT_END,B06_OCT_END,B07_OCT_END,B08_OCT_END,B8A_OCT_END,B09_OCT_END,B11_OCT_END,B12_OCT_END,CLM_OCT_END,B01_NOV_START,B02_NOV_START,B03_NOV_START,B04_NOV_START,B05_NOV_START,B06_NOV_START,B07_NOV_START,B08_NOV_START,B8A_NOV_START,B09_NOV_START,B11_NOV_START,B12_NOV_START,CLM_NOV_START,B01_NOV_END,B02_NOV_END,B03_NOV_END,B04_NOV_END,B05_NOV_END,B06_NOV_END,B07_NOV_END,B08_NOV_END,B8A_NOV_END,B09_NOV_END,B11_NOV_END,B12_NOV_END,CLM_NOV_END,y,field
0,18.154341,21.934084,29.180064,35.554662,43.747588,52.438907,57.289389,62.490354,63.221865,64.434084,68.397106,46.040193,253.770096,10.823151,13.385852,21.596463,29.223473,37.771704,46.858521,51.930868,57.065916,59.025723,59.684887,73.668810,49.313505,0.0,13.963023,15.408360,22.471061,29.371383,38.829582,47.680064,52.982315,56.434084,60.509646,60.676849,71.057878,46.557878,36.897106,15.144695,15.813505,23.000000,30.237942,37.426045,41.763666,45.440514,48.397106,51.731511,51.991961,79.366559,58.991961,0.0,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,255.000000,254.704180,255.000000,132.663987,111.138264,255.0,189.729904,166.000000,156.102894,150.096463,155.239550,150.897106,149.454984,148.377814,148.676849,152.442122,97.961415,65.845659,255.0,3.988746,9.366559,15.840836,15.077170,25.808682,42.319936,46.670418,51.408360,50.348875,50.612540,50.265273,32.673633,0.0,8.366559,9.265273,18.598071,14.300643,30.691318,63.106109,69.946945,73.424437,73.975884,73.988746,56.532154,32.744373,0.0,4.564309,8.829582,16.622186,11.213826,27.863344,64.485531,72.945338,75.901929,76.004823,76.848875,52.750804,30.342444,0.0,247.141479,230.686495,228.647910,228.053055,232.638264,227.472669,225.821543,228.831190,222.315113,233.003215,

In [8]:
# Prepare data
s_train = season_train.drop(['y', 'field'], 1)
s_test = season_test.drop('field', 1)
s_target = season_train.y
s_train.head()

,B01_PERIOD_0,B02_PERIOD_0,B03_PERIOD_0,B04_PERIOD_0,B05_PERIOD_0,B06_PERIOD_0,B07_PERIOD_0,B08_PERIOD_0,B8A_PERIOD_0,B09_PERIOD_0,B11_PERIOD_0,B12_PERIOD_0,CLM_PERIOD_0,B01_PERIOD_1,B02_PERIOD_1,B03_PERIOD_1,B04_PERIOD_1,B05_PERIOD_1,B06_PERIOD_1,B07_PERIOD_1,B08_PERIOD_1,B8A_PERIOD_1,B09_PERIOD_1,B11_PERIOD_1,B12_PERIOD_1,CLM_PERIOD_1,B01_PERIOD_2,B02_PERIOD_2,B03_PERIOD_2,B04_PERIOD_2,B05_PERIOD_2,B06_PERIOD_2,B07_PERIOD_2,B08_PERIOD_2,B8A_PERIOD_2,B09_PERIOD_2,B11_PERIOD_2,B12_PERIOD_2,CLM_PERIOD_2,B01_PERIOD_3,B02_PERIOD_3,B03_PERIOD_3,B04_PERIOD_3,B05_PERIOD_3,B06_PERIOD_3,B07_PERIOD_3,B08_PERIOD_3,B8A_PERIOD_3,B09_PERIOD_3,B11_PERIOD_3,B12_PERIOD_3,CLM_PERIOD_3,B01_PERIOD_4,B02_PERIOD_4,B03_PERIOD_4,B04_PERIOD_4,B05_PERIOD_4,B06_PERIOD_4,B07_PERIOD_4,B08_PERIOD_4,B8A_PERIOD_4,B09_PERIOD_4,B11_PERIOD_4,B12_PERIOD_4,CLM_PERIOD_4,B01_PERIOD_5,B02_PERIOD_5,B03_PERIOD_5,B04_PERIOD_5,B05_PERIOD_5,B06_PERIOD_5,B07_PERIOD_5,B08_PERIOD_5,B8A_PERIOD_5,B09_PERIOD_5,B11_PERIOD_5,B12_PERIOD_5,CLM_PERIOD_5,B01_PERIOD_6,B02_PERIOD_6,B03_PERIOD_6,B04_PERIOD_6,B05_PERIOD_6,B06_PERIOD_6,B07_PERIOD_6,B08_PERIOD_6,B8A_PERIOD_6,B09_PERIOD_6,B11_PERIOD_6,B12_PERIOD_6,CLM_PERIOD_6,B01_PERIOD_7,B02_PERIOD_7,B03_PERIOD_7,B04_PERIOD_7,B05_PERIOD_7,B06_PERIOD_7,B07_PERIOD_7,B08_PERIOD_7,B8A_PERIOD_7,B09_PERIOD_7,B11_PERIOD_7,B12_PERIOD_7,CLM_PERIOD_7,B01_PERIOD_8,B02_PERIOD_8,B03_PERIOD_8,B04_PERIOD_8,B05_PERIOD_8,B06_PERIOD_8,B07_PERIOD_8,B08_PERIOD_8,B8A_PERIOD_8,B09_PERIOD_8,B11_PERIOD_8,B12_PERIOD_8,CLM_PERIOD_8,B01_PERIOD_9,B02_PERIOD_9,B03_PERIOD_9,B04_PERIOD_9,B05_PERIOD_9,B06_PERIOD_9,B07_PERIOD_9,B08_PERIOD_9,B8A_PERIOD_9,B09_PERIOD_9,B11_PERIOD_9,B12_PERIOD_9,CLM_PERIOD_9,B01_PERIOD_10,B02_PERIOD_10,B03_PERIOD_10,B04_PERIOD_10,B05_PERIOD_10,B06_PERIOD_10,B07_PERIOD_10,B08_PERIOD_10,B8A_PERIOD_10,B09_PERIOD_10,B11_PERIOD_10,B12_PERIOD_10,CLM_PERIOD_10,B01_PERIOD_11,B02_PERIOD_11,B03_PERIOD_11,B04_PERIOD_11,B05_PERIOD_11,B06_PERIOD_11,B07_PERIOD_11,B08_PERIOD_11,B8A_PERIOD_11,B09_PERIOD_11,B11_PERIOD_11,B12_PERIOD_11,CLM_PERIOD_11,B01_PERIOD_12,B02_PERIOD_12,B03_PERIOD_12,B04_PERIOD_12,B05_PERIOD_12,B06_PERIOD_12,B07_PERIOD_12,B08_PERIOD_12,B8A_PERIOD_12,B09_PERIOD_12,B11_PERIOD_12,B12_PERIOD_12,CLM_PERIOD_12,B01_PERIOD_13,B02_PERIOD_13,B03_PERIOD_13,B04_PERIOD_13,B05_PERIOD_13,B06_PERIOD_13,B07_PERIOD_13,B08_PERIOD_13,B8A_PERIOD_13,B09_PERIOD_13,B11_PERIOD_13,B12_PERIOD_13,CLM_PERIOD_13,B01_PERIOD_14,B02_PERIOD_14,B03_PERIOD_14,B04_PERIOD_14,B05_PERIOD_14,B06_PERIOD_14,B07_PERIOD_14,B08_PERIOD_14,B8A_PERIOD_14,B09_PERIOD_14,B11_PERIOD_14,B12_PERIOD_14,CLM_PERIOD_14,B01_PERIOD_15,B02_PERIOD_15,B03_PERIOD_15,B04_PERIOD_15,B05_PERIOD_15,B06_PERIOD_15,B07_PERIOD_15,B08_PERIOD_15,B8A_PERIOD_15,B09_PERIOD_15,B11_PERIOD_15,B12_PERIOD_15,CLM_PERIOD_15,B01_PERIOD_16,B02_PERIOD_16,B03_PERIOD_16,B04_PERIOD_16,B05_PERIOD_16,B06_PERIOD_16,B07_PERIOD_16,B08_PERIOD_16,B8A_PERIOD_16,B09_PERIOD_16,B11_PERIOD_16,B12_PERIOD_16,CLM_PERIOD_16,B01_PERIOD_17,B02_PERIOD_17,B03_PERIOD_17,B04_PERIOD_17,B05_PERIOD_17,B06_PERIOD_17,B07_PERIOD_17,B08_PERIOD_17,B8A_PERIOD_17,B09_PERIOD_17,B11_PERIOD_17,B12_PERIOD_17,CLM_PERIOD_17,B01_PERIOD_18,B02_PERIOD_18,B03_PERIOD_18,B04_PERIOD_18,B05_PERIOD_18,B06_PERIOD_18,B07_PERIOD_18,B08_PERIOD_18,B8A_PERIOD_18,B09_PERIOD_18,B11_PERIOD_18,B12_PERIOD_18,CLM_PERIOD_18,B01_PERIOD_19,B02_PERIOD_19,B03_PERIOD_19,B04_PERIOD_19,B05_PERIOD_19,B06_PERIOD_19,B07_PERIOD_19,B08_PERIOD_19,B8A_PERIOD_19,B09_PERIOD_19,B11_PERIOD_19,B12_PERIOD_19,CLM_PERIOD_19,B01_PERIOD_20,B02_PERIOD_20,B03_PERIOD_20,B04_PERIOD_20,B05_PERIOD_20,B06_PERIOD_20,B07_PERIOD_20,B08_PERIOD_20,B8A_PERIOD_20,B09_PERIOD_20,B11_PERIOD_20,B12_PERIOD_20,CLM_PERIOD_20,B01_PERIOD_21,B02_PERIOD_21,B03_PERIOD_21,B04_PERIOD_21,B05_PERIOD_21,B06_PERIOD_21,B07_PERIOD_21,B08_PERIOD_21,B8A_PERIOD_21,B09_PERIOD_21,B11_PERIOD_21,B12_PERIOD_21,CLM_PERIOD_21,B01_PERIOD_22,B02_PERIOD_22,B03_PERIOD_22,B04_PERIOD_22,B05_PERIOD_22,B06_PERIOD_22,B07_PERIOD_22,B08_PERIOD_22,B8A_PERIOD_22

In [9]:
# Prepare data
m_train = season_train.drop(['y', 'field'], 1)
m_test = season_test.drop('field', 1)
m_target = season_train.y
m_train.head()

,B01_PERIOD_0,B02_PERIOD_0,B03_PERIOD_0,B04_PERIOD_0,B05_PERIOD_0,B06_PERIOD_0,B07_PERIOD_0,B08_PERIOD_0,B8A_PERIOD_0,B09_PERIOD_0,B11_PERIOD_0,B12_PERIOD_0,CLM_PERIOD_0,B01_PERIOD_1,B02_PERIOD_1,B03_PERIOD_1,B04_PERIOD_1,B05_PERIOD_1,B06_PERIOD_1,B07_PERIOD_1,B08_PERIOD_1,B8A_PERIOD_1,B09_PERIOD_1,B11_PERIOD_1,B12_PERIOD_1,CLM_PERIOD_1,B01_PERIOD_2,B02_PERIOD_2,B03_PERIOD_2,B04_PERIOD_2,B05_PERIOD_2,B06_PERIOD_2,B07_PERIOD_2,B08_PERIOD_2,B8A_PERIOD_2,B09_PERIOD_2,B11_PERIOD_2,B12_PERIOD_2,CLM_PERIOD_2,B01_PERIOD_3,B02_PERIOD_3,B03_PERIOD_3,B04_PERIOD_3,B05_PERIOD_3,B06_PERIOD_3,B07_PERIOD_3,B08_PERIOD_3,B8A_PERIOD_3,B09_PERIOD_3,B11_PERIOD_3,B12_PERIOD_3,CLM_PERIOD_3,B01_PERIOD_4,B02_PERIOD_4,B03_PERIOD_4,B04_PERIOD_4,B05_PERIOD_4,B06_PERIOD_4,B07_PERIOD_4,B08_PERIOD_4,B8A_PERIOD_4,B09_PERIOD_4,B11_PERIOD_4,B12_PERIOD_4,CLM_PERIOD_4,B01_PERIOD_5,B02_PERIOD_5,B03_PERIOD_5,B04_PERIOD_5,B05_PERIOD_5,B06_PERIOD_5,B07_PERIOD_5,B08_PERIOD_5,B8A_PERIOD_5,B09_PERIOD_5,B11_PERIOD_5,B12_PERIOD_5,CLM_PERIOD_5,B01_PERIOD_6,B02_PERIOD_6,B03_PERIOD_6,B04_PERIOD_6,B05_PERIOD_6,B06_PERIOD_6,B07_PERIOD_6,B08_PERIOD_6,B8A_PERIOD_6,B09_PERIOD_6,B11_PERIOD_6,B12_PERIOD_6,CLM_PERIOD_6,B01_PERIOD_7,B02_PERIOD_7,B03_PERIOD_7,B04_PERIOD_7,B05_PERIOD_7,B06_PERIOD_7,B07_PERIOD_7,B08_PERIOD_7,B8A_PERIOD_7,B09_PERIOD_7,B11_PERIOD_7,B12_PERIOD_7,CLM_PERIOD_7,B01_PERIOD_8,B02_PERIOD_8,B03_PERIOD_8,B04_PERIOD_8,B05_PERIOD_8,B06_PERIOD_8,B07_PERIOD_8,B08_PERIOD_8,B8A_PERIOD_8,B09_PERIOD_8,B11_PERIOD_8,B12_PERIOD_8,CLM_PERIOD_8,B01_PERIOD_9,B02_PERIOD_9,B03_PERIOD_9,B04_PERIOD_9,B05_PERIOD_9,B06_PERIOD_9,B07_PERIOD_9,B08_PERIOD_9,B8A_PERIOD_9,B09_PERIOD_9,B11_PERIOD_9,B12_PERIOD_9,CLM_PERIOD_9,B01_PERIOD_10,B02_PERIOD_10,B03_PERIOD_10,B04_PERIOD_10,B05_PERIOD_10,B06_PERIOD_10,B07_PERIOD_10,B08_PERIOD_10,B8A_PERIOD_10,B09_PERIOD_10,B11_PERIOD_10,B12_PERIOD_10,CLM_PERIOD_10,B01_PERIOD_11,B02_PERIOD_11,B03_PERIOD_11,B04_PERIOD_11,B05_PERIOD_11,B06_PERIOD_11,B07_PERIOD_11,B08_PERIOD_11,B8A_PERIOD_11,B09_PERIOD_11,B11_PERIOD_11,B12_PERIOD_11,CLM_PERIOD_11,B01_PERIOD_12,B02_PERIOD_12,B03_PERIOD_12,B04_PERIOD_12,B05_PERIOD_12,B06_PERIOD_12,B07_PERIOD_12,B08_PERIOD_12,B8A_PERIOD_12,B09_PERIOD_12,B11_PERIOD_12,B12_PERIOD_12,CLM_PERIOD_12,B01_PERIOD_13,B02_PERIOD_13,B03_PERIOD_13,B04_PERIOD_13,B05_PERIOD_13,B06_PERIOD_13,B07_PERIOD_13,B08_PERIOD_13,B8A_PERIOD_13,B09_PERIOD_13,B11_PERIOD_13,B12_PERIOD_13,CLM_PERIOD_13,B01_PERIOD_14,B02_PERIOD_14,B03_PERIOD_14,B04_PERIOD_14,B05_PERIOD_14,B06_PERIOD_14,B07_PERIOD_14,B08_PERIOD_14,B8A_PERIOD_14,B09_PERIOD_14,B11_PERIOD_14,B12_PERIOD_14,CLM_PERIOD_14,B01_PERIOD_15,B02_PERIOD_15,B03_PERIOD_15,B04_PERIOD_15,B05_PERIOD_15,B06_PERIOD_15,B07_PERIOD_15,B08_PERIOD_15,B8A_PERIOD_15,B09_PERIOD_15,B11_PERIOD_15,B12_PERIOD_15,CLM_PERIOD_15,B01_PERIOD_16,B02_PERIOD_16,B03_PERIOD_16,B04_PERIOD_16,B05_PERIOD_16,B06_PERIOD_16,B07_PERIOD_16,B08_PERIOD_16,B8A_PERIOD_16,B09_PERIOD_16,B11_PERIOD_16,B12_PERIOD_16,CLM_PERIOD_16,B01_PERIOD_17,B02_PERIOD_17,B03_PERIOD_17,B04_PERIOD_17,B05_PERIOD_17,B06_PERIOD_17,B07_PERIOD_17,B08_PERIOD_17,B8A_PERIOD_17,B09_PERIOD_17,B11_PERIOD_17,B12_PERIOD_17,CLM_PERIOD_17,B01_PERIOD_18,B02_PERIOD_18,B03_PERIOD_18,B04_PERIOD_18,B05_PERIOD_18,B06_PERIOD_18,B07_PERIOD_18,B08_PERIOD_18,B8A_PERIOD_18,B09_PERIOD_18,B11_PERIOD_18,B12_PERIOD_18,CLM_PERIOD_18,B01_PERIOD_19,B02_PERIOD_19,B03_PERIOD_19,B04_PERIOD_19,B05_PERIOD_19,B06_PERIOD_19,B07_PERIOD_19,B08_PERIOD_19,B8A_PERIOD_19,B09_PERIOD_19,B11_PERIOD_19,B12_PERIOD_19,CLM_PERIOD_19,B01_PERIOD_20,B02_PERIOD_20,B03_PERIOD_20,B04_PERIOD_20,B05_PERIOD_20,B06_PERIOD_20,B07_PERIOD_20,B08_PERIOD_20,B8A_PERIOD_20,B09_PERIOD_20,B11_PERIOD_20,B12_PERIOD_20,CLM_PERIOD_20,B01_PERIOD_21,B02_PERIOD_21,B03_PERIOD_21,B04_PERIOD_21,B05_PERIOD_21,B06_PERIOD_21,B07_PERIOD_21,B08_PERIOD_21,B8A_PERIOD_21,B09_PERIOD_21,B11_PERIOD_21,B12_PERIOD_21,CLM_PERIOD_21,B01_PERIOD_22,B02_PERIOD_22,B03_PERIOD_22,B04_PERIOD_22,B05_PERIOD_22,B06_PERIOD_22,B07_PERIOD_22,B08_PERIOD_22,B8A_PERIOD_22

### Handle Skewness

In [10]:
# Remove skewness by using square root
s_train, s_test, m_train, m_test = s_train**(1/2.0), s_test**(1/2.0), m_train**(1/2.0), m_test**(1/2.0)

# Modelling

## Pytorch - Season

In [11]:
# Prepare train data
df = s_train.fillna(s_train.median()).replace([np.inf, -np.inf],-999999)
df['target'] = s_target.copy()
df.head()

,B01_PERIOD_0,B02_PERIOD_0,B03_PERIOD_0,B04_PERIOD_0,B05_PERIOD_0,B06_PERIOD_0,B07_PERIOD_0,B08_PERIOD_0,B8A_PERIOD_0,B09_PERIOD_0,B11_PERIOD_0,B12_PERIOD_0,CLM_PERIOD_0,B01_PERIOD_1,B02_PERIOD_1,B03_PERIOD_1,B04_PERIOD_1,B05_PERIOD_1,B06_PERIOD_1,B07_PERIOD_1,B08_PERIOD_1,B8A_PERIOD_1,B09_PERIOD_1,B11_PERIOD_1,B12_PERIOD_1,CLM_PERIOD_1,B01_PERIOD_2,B02_PERIOD_2,B03_PERIOD_2,B04_PERIOD_2,B05_PERIOD_2,B06_PERIOD_2,B07_PERIOD_2,B08_PERIOD_2,B8A_PERIOD_2,B09_PERIOD_2,B11_PERIOD_2,B12_PERIOD_2,CLM_PERIOD_2,B01_PERIOD_3,B02_PERIOD_3,B03_PERIOD_3,B04_PERIOD_3,B05_PERIOD_3,B06_PERIOD_3,B07_PERIOD_3,B08_PERIOD_3,B8A_PERIOD_3,B09_PERIOD_3,B11_PERIOD_3,B12_PERIOD_3,CLM_PERIOD_3,B01_PERIOD_4,B02_PERIOD_4,B03_PERIOD_4,B04_PERIOD_4,B05_PERIOD_4,B06_PERIOD_4,B07_PERIOD_4,B08_PERIOD_4,B8A_PERIOD_4,B09_PERIOD_4,B11_PERIOD_4,B12_PERIOD_4,CLM_PERIOD_4,B01_PERIOD_5,B02_PERIOD_5,B03_PERIOD_5,B04_PERIOD_5,B05_PERIOD_5,B06_PERIOD_5,B07_PERIOD_5,B08_PERIOD_5,B8A_PERIOD_5,B09_PERIOD_5,B11_PERIOD_5,B12_PERIOD_5,CLM_PERIOD_5,B01_PERIOD_6,B02_PERIOD_6,B03_PERIOD_6,B04_PERIOD_6,B05_PERIOD_6,B06_PERIOD_6,B07_PERIOD_6,B08_PERIOD_6,B8A_PERIOD_6,B09_PERIOD_6,B11_PERIOD_6,B12_PERIOD_6,CLM_PERIOD_6,B01_PERIOD_7,B02_PERIOD_7,B03_PERIOD_7,B04_PERIOD_7,B05_PERIOD_7,B06_PERIOD_7,B07_PERIOD_7,B08_PERIOD_7,B8A_PERIOD_7,B09_PERIOD_7,B11_PERIOD_7,B12_PERIOD_7,CLM_PERIOD_7,B01_PERIOD_8,B02_PERIOD_8,B03_PERIOD_8,B04_PERIOD_8,B05_PERIOD_8,B06_PERIOD_8,B07_PERIOD_8,B08_PERIOD_8,B8A_PERIOD_8,B09_PERIOD_8,B11_PERIOD_8,B12_PERIOD_8,CLM_PERIOD_8,B01_PERIOD_9,B02_PERIOD_9,B03_PERIOD_9,B04_PERIOD_9,B05_PERIOD_9,B06_PERIOD_9,B07_PERIOD_9,B08_PERIOD_9,B8A_PERIOD_9,B09_PERIOD_9,B11_PERIOD_9,B12_PERIOD_9,CLM_PERIOD_9,B01_PERIOD_10,B02_PERIOD_10,B03_PERIOD_10,B04_PERIOD_10,B05_PERIOD_10,B06_PERIOD_10,B07_PERIOD_10,B08_PERIOD_10,B8A_PERIOD_10,B09_PERIOD_10,B11_PERIOD_10,B12_PERIOD_10,CLM_PERIOD_10,B01_PERIOD_11,B02_PERIOD_11,B03_PERIOD_11,B04_PERIOD_11,B05_PERIOD_11,B06_PERIOD_11,B07_PERIOD_11,B08_PERIOD_11,B8A_PERIOD_11,B09_PERIOD_11,B11_PERIOD_11,B12_PERIOD_11,CLM_PERIOD_11,B01_PERIOD_12,B02_PERIOD_12,B03_PERIOD_12,B04_PERIOD_12,B05_PERIOD_12,B06_PERIOD_12,B07_PERIOD_12,B08_PERIOD_12,B8A_PERIOD_12,B09_PERIOD_12,B11_PERIOD_12,B12_PERIOD_12,CLM_PERIOD_12,B01_PERIOD_13,B02_PERIOD_13,B03_PERIOD_13,B04_PERIOD_13,B05_PERIOD_13,B06_PERIOD_13,B07_PERIOD_13,B08_PERIOD_13,B8A_PERIOD_13,B09_PERIOD_13,B11_PERIOD_13,B12_PERIOD_13,CLM_PERIOD_13,B01_PERIOD_14,B02_PERIOD_14,B03_PERIOD_14,B04_PERIOD_14,B05_PERIOD_14,B06_PERIOD_14,B07_PERIOD_14,B08_PERIOD_14,B8A_PERIOD_14,B09_PERIOD_14,B11_PERIOD_14,B12_PERIOD_14,CLM_PERIOD_14,B01_PERIOD_15,B02_PERIOD_15,B03_PERIOD_15,B04_PERIOD_15,B05_PERIOD_15,B06_PERIOD_15,B07_PERIOD_15,B08_PERIOD_15,B8A_PERIOD_15,B09_PERIOD_15,B11_PERIOD_15,B12_PERIOD_15,CLM_PERIOD_15,B01_PERIOD_16,B02_PERIOD_16,B03_PERIOD_16,B04_PERIOD_16,B05_PERIOD_16,B06_PERIOD_16,B07_PERIOD_16,B08_PERIOD_16,B8A_PERIOD_16,B09_PERIOD_16,B11_PERIOD_16,B12_PERIOD_16,CLM_PERIOD_16,B01_PERIOD_17,B02_PERIOD_17,B03_PERIOD_17,B04_PERIOD_17,B05_PERIOD_17,B06_PERIOD_17,B07_PERIOD_17,B08_PERIOD_17,B8A_PERIOD_17,B09_PERIOD_17,B11_PERIOD_17,B12_PERIOD_17,CLM_PERIOD_17,B01_PERIOD_18,B02_PERIOD_18,B03_PERIOD_18,B04_PERIOD_18,B05_PERIOD_18,B06_PERIOD_18,B07_PERIOD_18,B08_PERIOD_18,B8A_PERIOD_18,B09_PERIOD_18,B11_PERIOD_18,B12_PERIOD_18,CLM_PERIOD_18,B01_PERIOD_19,B02_PERIOD_19,B03_PERIOD_19,B04_PERIOD_19,B05_PERIOD_19,B06_PERIOD_19,B07_PERIOD_19,B08_PERIOD_19,B8A_PERIOD_19,B09_PERIOD_19,B11_PERIOD_19,B12_PERIOD_19,CLM_PERIOD_19,B01_PERIOD_20,B02_PERIOD_20,B03_PERIOD_20,B04_PERIOD_20,B05_PERIOD_20,B06_PERIOD_20,B07_PERIOD_20,B08_PERIOD_20,B8A_PERIOD_20,B09_PERIOD_20,B11_PERIOD_20,B12_PERIOD_20,CLM_PERIOD_20,B01_PERIOD_21,B02_PERIOD_21,B03_PERIOD_21,B04_PERIOD_21,B05_PERIOD_21,B06_PERIOD_21,B07_PERIOD_21,B08_PERIOD_21,B8A_PERIOD_21,B09_PERIOD_21,B11_PERIOD_21,B12_PERIOD_21,CLM_PERIOD_21,B01_PERIOD_22,B02_PERIOD_22,B03_PERIOD_22,B04_PERIOD_22,B05_PERIOD_22,B06_PERIOD_22,B07_PERIOD_22,B08_PERIOD_22,B8A_PERIOD_22

In [12]:
# Prepare test data
tess = s_test.fillna(s_test.median()).replace([np.inf, -np.inf], -999999)
tess['target'] = 1
tess.head()

,B01_PERIOD_0,B02_PERIOD_0,B03_PERIOD_0,B04_PERIOD_0,B05_PERIOD_0,B06_PERIOD_0,B07_PERIOD_0,B08_PERIOD_0,B8A_PERIOD_0,B09_PERIOD_0,B11_PERIOD_0,B12_PERIOD_0,CLM_PERIOD_0,B01_PERIOD_1,B02_PERIOD_1,B03_PERIOD_1,B04_PERIOD_1,B05_PERIOD_1,B06_PERIOD_1,B07_PERIOD_1,B08_PERIOD_1,B8A_PERIOD_1,B09_PERIOD_1,B11_PERIOD_1,B12_PERIOD_1,CLM_PERIOD_1,B01_PERIOD_2,B02_PERIOD_2,B03_PERIOD_2,B04_PERIOD_2,B05_PERIOD_2,B06_PERIOD_2,B07_PERIOD_2,B08_PERIOD_2,B8A_PERIOD_2,B09_PERIOD_2,B11_PERIOD_2,B12_PERIOD_2,CLM_PERIOD_2,B01_PERIOD_3,B02_PERIOD_3,B03_PERIOD_3,B04_PERIOD_3,B05_PERIOD_3,B06_PERIOD_3,B07_PERIOD_3,B08_PERIOD_3,B8A_PERIOD_3,B09_PERIOD_3,B11_PERIOD_3,B12_PERIOD_3,CLM_PERIOD_3,B01_PERIOD_4,B02_PERIOD_4,B03_PERIOD_4,B04_PERIOD_4,B05_PERIOD_4,B06_PERIOD_4,B07_PERIOD_4,B08_PERIOD_4,B8A_PERIOD_4,B09_PERIOD_4,B11_PERIOD_4,B12_PERIOD_4,CLM_PERIOD_4,B01_PERIOD_5,B02_PERIOD_5,B03_PERIOD_5,B04_PERIOD_5,B05_PERIOD_5,B06_PERIOD_5,B07_PERIOD_5,B08_PERIOD_5,B8A_PERIOD_5,B09_PERIOD_5,B11_PERIOD_5,B12_PERIOD_5,CLM_PERIOD_5,B01_PERIOD_6,B02_PERIOD_6,B03_PERIOD_6,B04_PERIOD_6,B05_PERIOD_6,B06_PERIOD_6,B07_PERIOD_6,B08_PERIOD_6,B8A_PERIOD_6,B09_PERIOD_6,B11_PERIOD_6,B12_PERIOD_6,CLM_PERIOD_6,B01_PERIOD_7,B02_PERIOD_7,B03_PERIOD_7,B04_PERIOD_7,B05_PERIOD_7,B06_PERIOD_7,B07_PERIOD_7,B08_PERIOD_7,B8A_PERIOD_7,B09_PERIOD_7,B11_PERIOD_7,B12_PERIOD_7,CLM_PERIOD_7,B01_PERIOD_8,B02_PERIOD_8,B03_PERIOD_8,B04_PERIOD_8,B05_PERIOD_8,B06_PERIOD_8,B07_PERIOD_8,B08_PERIOD_8,B8A_PERIOD_8,B09_PERIOD_8,B11_PERIOD_8,B12_PERIOD_8,CLM_PERIOD_8,B01_PERIOD_9,B02_PERIOD_9,B03_PERIOD_9,B04_PERIOD_9,B05_PERIOD_9,B06_PERIOD_9,B07_PERIOD_9,B08_PERIOD_9,B8A_PERIOD_9,B09_PERIOD_9,B11_PERIOD_9,B12_PERIOD_9,CLM_PERIOD_9,B01_PERIOD_10,B02_PERIOD_10,B03_PERIOD_10,B04_PERIOD_10,B05_PERIOD_10,B06_PERIOD_10,B07_PERIOD_10,B08_PERIOD_10,B8A_PERIOD_10,B09_PERIOD_10,B11_PERIOD_10,B12_PERIOD_10,CLM_PERIOD_10,B01_PERIOD_11,B02_PERIOD_11,B03_PERIOD_11,B04_PERIOD_11,B05_PERIOD_11,B06_PERIOD_11,B07_PERIOD_11,B08_PERIOD_11,B8A_PERIOD_11,B09_PERIOD_11,B11_PERIOD_11,B12_PERIOD_11,CLM_PERIOD_11,B01_PERIOD_12,B02_PERIOD_12,B03_PERIOD_12,B04_PERIOD_12,B05_PERIOD_12,B06_PERIOD_12,B07_PERIOD_12,B08_PERIOD_12,B8A_PERIOD_12,B09_PERIOD_12,B11_PERIOD_12,B12_PERIOD_12,CLM_PERIOD_12,B01_PERIOD_13,B02_PERIOD_13,B03_PERIOD_13,B04_PERIOD_13,B05_PERIOD_13,B06_PERIOD_13,B07_PERIOD_13,B08_PERIOD_13,B8A_PERIOD_13,B09_PERIOD_13,B11_PERIOD_13,B12_PERIOD_13,CLM_PERIOD_13,B01_PERIOD_14,B02_PERIOD_14,B03_PERIOD_14,B04_PERIOD_14,B05_PERIOD_14,B06_PERIOD_14,B07_PERIOD_14,B08_PERIOD_14,B8A_PERIOD_14,B09_PERIOD_14,B11_PERIOD_14,B12_PERIOD_14,CLM_PERIOD_14,B01_PERIOD_15,B02_PERIOD_15,B03_PERIOD_15,B04_PERIOD_15,B05_PERIOD_15,B06_PERIOD_15,B07_PERIOD_15,B08_PERIOD_15,B8A_PERIOD_15,B09_PERIOD_15,B11_PERIOD_15,B12_PERIOD_15,CLM_PERIOD_15,B01_PERIOD_16,B02_PERIOD_16,B03_PERIOD_16,B04_PERIOD_16,B05_PERIOD_16,B06_PERIOD_16,B07_PERIOD_16,B08_PERIOD_16,B8A_PERIOD_16,B09_PERIOD_16,B11_PERIOD_16,B12_PERIOD_16,CLM_PERIOD_16,B01_PERIOD_17,B02_PERIOD_17,B03_PERIOD_17,B04_PERIOD_17,B05_PERIOD_17,B06_PERIOD_17,B07_PERIOD_17,B08_PERIOD_17,B8A_PERIOD_17,B09_PERIOD_17,B11_PERIOD_17,B12_PERIOD_17,CLM_PERIOD_17,B01_PERIOD_18,B02_PERIOD_18,B03_PERIOD_18,B04_PERIOD_18,B05_PERIOD_18,B06_PERIOD_18,B07_PERIOD_18,B08_PERIOD_18,B8A_PERIOD_18,B09_PERIOD_18,B11_PERIOD_18,B12_PERIOD_18,CLM_PERIOD_18,B01_PERIOD_19,B02_PERIOD_19,B03_PERIOD_19,B04_PERIOD_19,B05_PERIOD_19,B06_PERIOD_19,B07_PERIOD_19,B08_PERIOD_19,B8A_PERIOD_19,B09_PERIOD_19,B11_PERIOD_19,B12_PERIOD_19,CLM_PERIOD_19,B01_PERIOD_20,B02_PERIOD_20,B03_PERIOD_20,B04_PERIOD_20,B05_PERIOD_20,B06_PERIOD_20,B07_PERIOD_20,B08_PERIOD_20,B8A_PERIOD_20,B09_PERIOD_20,B11_PERIOD_20,B12_PERIOD_20,CLM_PERIOD_20,B01_PERIOD_21,B02_PERIOD_21,B03_PERIOD_21,B04_PERIOD_21,B05_PERIOD_21,B06_PERIOD_21,B07_PERIOD_21,B08_PERIOD_21,B8A_PERIOD_21,B09_PERIOD_21,B11_PERIOD_21,B12_PERIOD_21,CLM_PERIOD_21,B01_PERIOD_22,B02_PERIOD_22,B03_PERIOD_22,B04_PERIOD_22,B05_PERIOD_22,B06_PERIOD_22,B07_PERIOD_22,B08_PERIOD_22,B8A_PERIOD_22

#### Train Model

In [ ]:
%%time
# Select X and y features
X = df.copy()
y = df.target

# Lists to store predictions and losses
season_predictions = []
season_losses = []

# Stratified Validation
folds = StratifiedKFold(n_splits = 20, shuffle = True, random_state=seed)

for i,( train_index, test_index) in enumerate(folds.split(X, y)):
  X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

  # Parameters and configuration
  data_config = DataConfig(
      target=['target'],
      continuous_cols = s_test.columns.tolist(),
      categorical_cols = [],
      normalize_continuous_features = True,
      # continuous_feature_transform = 'box-cox',
      num_workers = 4
  )

  trainer_config = TrainerConfig(
      auto_lr_find=True, 
      batch_size=4096,
      max_epochs=500,
      gpus=-1, 
      load_best = True,
      early_stopping_min_delta = 0.01,
      early_stopping_patience = 20,
      profiler = None
  )
  optimizer_config = OptimizerConfig() 

  model_config = CategoryEmbeddingModelConfig(
      task="classification",
      layers="4096-1024-512", 
      activation="LeakyReLU", 
      use_batch_norm = False,
      dropout = 0.5,
      seed = 5050
  )

  # Instantiate model
  tabular_model = TabularModel(
      data_config=data_config,
      model_config=model_config,
      optimizer_config=optimizer_config,
      trainer_config=trainer_config,
  )

  # Train Model
  tabular_model.fit(train=X_train, validation=X_test)
  result = tabular_model.evaluate(X_test)

  # Print loss per fold
  print(f'\n \033[92m Fold:{i}__{log_loss(X_test.target, tabular_model.predict(X_test).iloc[:,X_test.shape[1]:-1].values)}')

  #Append predictions and losses
  season_losses.append(log_loss(X_test.target, tabular_model.predict(X_test).iloc[:,X_test.shape[1]:-1].values))
  season_predictions.append(tabular_model.predict(tess).iloc[:,tess.shape[1]:-1].values)

# Print CV score
print(f'\n \033[92m Mean Loss: {np.mean(season_losses)}')

Global seed set to 5050
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 650   
2 | backbone               | FeedForwardBackbone | 6.1 M 
3 | output_layer           | Linear              | 4.6 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
6.1 M     Trainable params
0         Non-trainable params
6.1 M     Total params
24.243    Total estimated model params size (MB)
Global seed set to 5050


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


LR finder stopped early after 90 steps due to diverging loss.
Restored states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 0.0004365158322401656
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 650   
2 | backbone               | FeedForwardBackbone | 6.1 M 
3 | output_layer           | Linear              | 4.6 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
6.1 M     Trainable params
0         Non-trainable params
6.1 M     Total params
24.243    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [14]:
def sub_creator(mpreds, name): 
  ss_cols = ['Field ID', 'Crop_Lucerne/Medics', 'Crop_Planted pastures (perennial)', 'Crop_Fallow', 'Crop_Wine grapes', 'Crop_Weeds',\
              'Crop_Small grain grazing', 'Crop_Wheat', 'Crop_Canola',  'Crop_Rooibos']
  ss = pd.DataFrame(mpreds, columns = ss_cols[1:])
  ss['Field ID'] = season_test.field.astype(int)
  ss = ss[ss_cols]
  ss.to_csv(f'{name}.csv', index = False)
  ss.head()

## Pytorch - Months

In [15]:
# Prepare train data
df = m_train.fillna(m_train.median()).replace([np.inf, -np.inf],-999999)
df['target'] = m_target.copy()
df.head()

,B01_PERIOD_0,B02_PERIOD_0,B03_PERIOD_0,B04_PERIOD_0,B05_PERIOD_0,B06_PERIOD_0,B07_PERIOD_0,B08_PERIOD_0,B8A_PERIOD_0,B09_PERIOD_0,B11_PERIOD_0,B12_PERIOD_0,CLM_PERIOD_0,B01_PERIOD_1,B02_PERIOD_1,B03_PERIOD_1,B04_PERIOD_1,B05_PERIOD_1,B06_PERIOD_1,B07_PERIOD_1,B08_PERIOD_1,B8A_PERIOD_1,B09_PERIOD_1,B11_PERIOD_1,B12_PERIOD_1,CLM_PERIOD_1,B01_PERIOD_2,B02_PERIOD_2,B03_PERIOD_2,B04_PERIOD_2,B05_PERIOD_2,B06_PERIOD_2,B07_PERIOD_2,B08_PERIOD_2,B8A_PERIOD_2,B09_PERIOD_2,B11_PERIOD_2,B12_PERIOD_2,CLM_PERIOD_2,B01_PERIOD_3,B02_PERIOD_3,B03_PERIOD_3,B04_PERIOD_3,B05_PERIOD_3,B06_PERIOD_3,B07_PERIOD_3,B08_PERIOD_3,B8A_PERIOD_3,B09_PERIOD_3,B11_PERIOD_3,B12_PERIOD_3,CLM_PERIOD_3,B01_PERIOD_4,B02_PERIOD_4,B03_PERIOD_4,B04_PERIOD_4,B05_PERIOD_4,B06_PERIOD_4,B07_PERIOD_4,B08_PERIOD_4,B8A_PERIOD_4,B09_PERIOD_4,B11_PERIOD_4,B12_PERIOD_4,CLM_PERIOD_4,B01_PERIOD_5,B02_PERIOD_5,B03_PERIOD_5,B04_PERIOD_5,B05_PERIOD_5,B06_PERIOD_5,B07_PERIOD_5,B08_PERIOD_5,B8A_PERIOD_5,B09_PERIOD_5,B11_PERIOD_5,B12_PERIOD_5,CLM_PERIOD_5,B01_PERIOD_6,B02_PERIOD_6,B03_PERIOD_6,B04_PERIOD_6,B05_PERIOD_6,B06_PERIOD_6,B07_PERIOD_6,B08_PERIOD_6,B8A_PERIOD_6,B09_PERIOD_6,B11_PERIOD_6,B12_PERIOD_6,CLM_PERIOD_6,B01_PERIOD_7,B02_PERIOD_7,B03_PERIOD_7,B04_PERIOD_7,B05_PERIOD_7,B06_PERIOD_7,B07_PERIOD_7,B08_PERIOD_7,B8A_PERIOD_7,B09_PERIOD_7,B11_PERIOD_7,B12_PERIOD_7,CLM_PERIOD_7,B01_PERIOD_8,B02_PERIOD_8,B03_PERIOD_8,B04_PERIOD_8,B05_PERIOD_8,B06_PERIOD_8,B07_PERIOD_8,B08_PERIOD_8,B8A_PERIOD_8,B09_PERIOD_8,B11_PERIOD_8,B12_PERIOD_8,CLM_PERIOD_8,B01_PERIOD_9,B02_PERIOD_9,B03_PERIOD_9,B04_PERIOD_9,B05_PERIOD_9,B06_PERIOD_9,B07_PERIOD_9,B08_PERIOD_9,B8A_PERIOD_9,B09_PERIOD_9,B11_PERIOD_9,B12_PERIOD_9,CLM_PERIOD_9,B01_PERIOD_10,B02_PERIOD_10,B03_PERIOD_10,B04_PERIOD_10,B05_PERIOD_10,B06_PERIOD_10,B07_PERIOD_10,B08_PERIOD_10,B8A_PERIOD_10,B09_PERIOD_10,B11_PERIOD_10,B12_PERIOD_10,CLM_PERIOD_10,B01_PERIOD_11,B02_PERIOD_11,B03_PERIOD_11,B04_PERIOD_11,B05_PERIOD_11,B06_PERIOD_11,B07_PERIOD_11,B08_PERIOD_11,B8A_PERIOD_11,B09_PERIOD_11,B11_PERIOD_11,B12_PERIOD_11,CLM_PERIOD_11,B01_PERIOD_12,B02_PERIOD_12,B03_PERIOD_12,B04_PERIOD_12,B05_PERIOD_12,B06_PERIOD_12,B07_PERIOD_12,B08_PERIOD_12,B8A_PERIOD_12,B09_PERIOD_12,B11_PERIOD_12,B12_PERIOD_12,CLM_PERIOD_12,B01_PERIOD_13,B02_PERIOD_13,B03_PERIOD_13,B04_PERIOD_13,B05_PERIOD_13,B06_PERIOD_13,B07_PERIOD_13,B08_PERIOD_13,B8A_PERIOD_13,B09_PERIOD_13,B11_PERIOD_13,B12_PERIOD_13,CLM_PERIOD_13,B01_PERIOD_14,B02_PERIOD_14,B03_PERIOD_14,B04_PERIOD_14,B05_PERIOD_14,B06_PERIOD_14,B07_PERIOD_14,B08_PERIOD_14,B8A_PERIOD_14,B09_PERIOD_14,B11_PERIOD_14,B12_PERIOD_14,CLM_PERIOD_14,B01_PERIOD_15,B02_PERIOD_15,B03_PERIOD_15,B04_PERIOD_15,B05_PERIOD_15,B06_PERIOD_15,B07_PERIOD_15,B08_PERIOD_15,B8A_PERIOD_15,B09_PERIOD_15,B11_PERIOD_15,B12_PERIOD_15,CLM_PERIOD_15,B01_PERIOD_16,B02_PERIOD_16,B03_PERIOD_16,B04_PERIOD_16,B05_PERIOD_16,B06_PERIOD_16,B07_PERIOD_16,B08_PERIOD_16,B8A_PERIOD_16,B09_PERIOD_16,B11_PERIOD_16,B12_PERIOD_16,CLM_PERIOD_16,B01_PERIOD_17,B02_PERIOD_17,B03_PERIOD_17,B04_PERIOD_17,B05_PERIOD_17,B06_PERIOD_17,B07_PERIOD_17,B08_PERIOD_17,B8A_PERIOD_17,B09_PERIOD_17,B11_PERIOD_17,B12_PERIOD_17,CLM_PERIOD_17,B01_PERIOD_18,B02_PERIOD_18,B03_PERIOD_18,B04_PERIOD_18,B05_PERIOD_18,B06_PERIOD_18,B07_PERIOD_18,B08_PERIOD_18,B8A_PERIOD_18,B09_PERIOD_18,B11_PERIOD_18,B12_PERIOD_18,CLM_PERIOD_18,B01_PERIOD_19,B02_PERIOD_19,B03_PERIOD_19,B04_PERIOD_19,B05_PERIOD_19,B06_PERIOD_19,B07_PERIOD_19,B08_PERIOD_19,B8A_PERIOD_19,B09_PERIOD_19,B11_PERIOD_19,B12_PERIOD_19,CLM_PERIOD_19,B01_PERIOD_20,B02_PERIOD_20,B03_PERIOD_20,B04_PERIOD_20,B05_PERIOD_20,B06_PERIOD_20,B07_PERIOD_20,B08_PERIOD_20,B8A_PERIOD_20,B09_PERIOD_20,B11_PERIOD_20,B12_PERIOD_20,CLM_PERIOD_20,B01_PERIOD_21,B02_PERIOD_21,B03_PERIOD_21,B04_PERIOD_21,B05_PERIOD_21,B06_PERIOD_21,B07_PERIOD_21,B08_PERIOD_21,B8A_PERIOD_21,B09_PERIOD_21,B11_PERIOD_21,B12_PERIOD_21,CLM_PERIOD_21,B01_PERIOD_22,B02_PERIOD_22,B03_PERIOD_22,B04_PERIOD_22,B05_PERIOD_22,B06_PERIOD_22,B07_PERIOD_22,B08_PERIOD_22,B8A_PERIOD_22

In [16]:
# Prepare test data
tess = m_test.fillna(m_test.median()).replace([np.inf, -np.inf], -999999)
tess['target'] = 1
tess.head()

,B01_PERIOD_0,B02_PERIOD_0,B03_PERIOD_0,B04_PERIOD_0,B05_PERIOD_0,B06_PERIOD_0,B07_PERIOD_0,B08_PERIOD_0,B8A_PERIOD_0,B09_PERIOD_0,B11_PERIOD_0,B12_PERIOD_0,CLM_PERIOD_0,B01_PERIOD_1,B02_PERIOD_1,B03_PERIOD_1,B04_PERIOD_1,B05_PERIOD_1,B06_PERIOD_1,B07_PERIOD_1,B08_PERIOD_1,B8A_PERIOD_1,B09_PERIOD_1,B11_PERIOD_1,B12_PERIOD_1,CLM_PERIOD_1,B01_PERIOD_2,B02_PERIOD_2,B03_PERIOD_2,B04_PERIOD_2,B05_PERIOD_2,B06_PERIOD_2,B07_PERIOD_2,B08_PERIOD_2,B8A_PERIOD_2,B09_PERIOD_2,B11_PERIOD_2,B12_PERIOD_2,CLM_PERIOD_2,B01_PERIOD_3,B02_PERIOD_3,B03_PERIOD_3,B04_PERIOD_3,B05_PERIOD_3,B06_PERIOD_3,B07_PERIOD_3,B08_PERIOD_3,B8A_PERIOD_3,B09_PERIOD_3,B11_PERIOD_3,B12_PERIOD_3,CLM_PERIOD_3,B01_PERIOD_4,B02_PERIOD_4,B03_PERIOD_4,B04_PERIOD_4,B05_PERIOD_4,B06_PERIOD_4,B07_PERIOD_4,B08_PERIOD_4,B8A_PERIOD_4,B09_PERIOD_4,B11_PERIOD_4,B12_PERIOD_4,CLM_PERIOD_4,B01_PERIOD_5,B02_PERIOD_5,B03_PERIOD_5,B04_PERIOD_5,B05_PERIOD_5,B06_PERIOD_5,B07_PERIOD_5,B08_PERIOD_5,B8A_PERIOD_5,B09_PERIOD_5,B11_PERIOD_5,B12_PERIOD_5,CLM_PERIOD_5,B01_PERIOD_6,B02_PERIOD_6,B03_PERIOD_6,B04_PERIOD_6,B05_PERIOD_6,B06_PERIOD_6,B07_PERIOD_6,B08_PERIOD_6,B8A_PERIOD_6,B09_PERIOD_6,B11_PERIOD_6,B12_PERIOD_6,CLM_PERIOD_6,B01_PERIOD_7,B02_PERIOD_7,B03_PERIOD_7,B04_PERIOD_7,B05_PERIOD_7,B06_PERIOD_7,B07_PERIOD_7,B08_PERIOD_7,B8A_PERIOD_7,B09_PERIOD_7,B11_PERIOD_7,B12_PERIOD_7,CLM_PERIOD_7,B01_PERIOD_8,B02_PERIOD_8,B03_PERIOD_8,B04_PERIOD_8,B05_PERIOD_8,B06_PERIOD_8,B07_PERIOD_8,B08_PERIOD_8,B8A_PERIOD_8,B09_PERIOD_8,B11_PERIOD_8,B12_PERIOD_8,CLM_PERIOD_8,B01_PERIOD_9,B02_PERIOD_9,B03_PERIOD_9,B04_PERIOD_9,B05_PERIOD_9,B06_PERIOD_9,B07_PERIOD_9,B08_PERIOD_9,B8A_PERIOD_9,B09_PERIOD_9,B11_PERIOD_9,B12_PERIOD_9,CLM_PERIOD_9,B01_PERIOD_10,B02_PERIOD_10,B03_PERIOD_10,B04_PERIOD_10,B05_PERIOD_10,B06_PERIOD_10,B07_PERIOD_10,B08_PERIOD_10,B8A_PERIOD_10,B09_PERIOD_10,B11_PERIOD_10,B12_PERIOD_10,CLM_PERIOD_10,B01_PERIOD_11,B02_PERIOD_11,B03_PERIOD_11,B04_PERIOD_11,B05_PERIOD_11,B06_PERIOD_11,B07_PERIOD_11,B08_PERIOD_11,B8A_PERIOD_11,B09_PERIOD_11,B11_PERIOD_11,B12_PERIOD_11,CLM_PERIOD_11,B01_PERIOD_12,B02_PERIOD_12,B03_PERIOD_12,B04_PERIOD_12,B05_PERIOD_12,B06_PERIOD_12,B07_PERIOD_12,B08_PERIOD_12,B8A_PERIOD_12,B09_PERIOD_12,B11_PERIOD_12,B12_PERIOD_12,CLM_PERIOD_12,B01_PERIOD_13,B02_PERIOD_13,B03_PERIOD_13,B04_PERIOD_13,B05_PERIOD_13,B06_PERIOD_13,B07_PERIOD_13,B08_PERIOD_13,B8A_PERIOD_13,B09_PERIOD_13,B11_PERIOD_13,B12_PERIOD_13,CLM_PERIOD_13,B01_PERIOD_14,B02_PERIOD_14,B03_PERIOD_14,B04_PERIOD_14,B05_PERIOD_14,B06_PERIOD_14,B07_PERIOD_14,B08_PERIOD_14,B8A_PERIOD_14,B09_PERIOD_14,B11_PERIOD_14,B12_PERIOD_14,CLM_PERIOD_14,B01_PERIOD_15,B02_PERIOD_15,B03_PERIOD_15,B04_PERIOD_15,B05_PERIOD_15,B06_PERIOD_15,B07_PERIOD_15,B08_PERIOD_15,B8A_PERIOD_15,B09_PERIOD_15,B11_PERIOD_15,B12_PERIOD_15,CLM_PERIOD_15,B01_PERIOD_16,B02_PERIOD_16,B03_PERIOD_16,B04_PERIOD_16,B05_PERIOD_16,B06_PERIOD_16,B07_PERIOD_16,B08_PERIOD_16,B8A_PERIOD_16,B09_PERIOD_16,B11_PERIOD_16,B12_PERIOD_16,CLM_PERIOD_16,B01_PERIOD_17,B02_PERIOD_17,B03_PERIOD_17,B04_PERIOD_17,B05_PERIOD_17,B06_PERIOD_17,B07_PERIOD_17,B08_PERIOD_17,B8A_PERIOD_17,B09_PERIOD_17,B11_PERIOD_17,B12_PERIOD_17,CLM_PERIOD_17,B01_PERIOD_18,B02_PERIOD_18,B03_PERIOD_18,B04_PERIOD_18,B05_PERIOD_18,B06_PERIOD_18,B07_PERIOD_18,B08_PERIOD_18,B8A_PERIOD_18,B09_PERIOD_18,B11_PERIOD_18,B12_PERIOD_18,CLM_PERIOD_18,B01_PERIOD_19,B02_PERIOD_19,B03_PERIOD_19,B04_PERIOD_19,B05_PERIOD_19,B06_PERIOD_19,B07_PERIOD_19,B08_PERIOD_19,B8A_PERIOD_19,B09_PERIOD_19,B11_PERIOD_19,B12_PERIOD_19,CLM_PERIOD_19,B01_PERIOD_20,B02_PERIOD_20,B03_PERIOD_20,B04_PERIOD_20,B05_PERIOD_20,B06_PERIOD_20,B07_PERIOD_20,B08_PERIOD_20,B8A_PERIOD_20,B09_PERIOD_20,B11_PERIOD_20,B12_PERIOD_20,CLM_PERIOD_20,B01_PERIOD_21,B02_PERIOD_21,B03_PERIOD_21,B04_PERIOD_21,B05_PERIOD_21,B06_PERIOD_21,B07_PERIOD_21,B08_PERIOD_21,B8A_PERIOD_21,B09_PERIOD_21,B11_PERIOD_21,B12_PERIOD_21,CLM_PERIOD_21,B01_PERIOD_22,B02_PERIOD_22,B03_PERIOD_22,B04_PERIOD_22,B05_PERIOD_22,B06_PERIOD_22,B07_PERIOD_22,B08_PERIOD_22,B8A_PERIOD_22

### Train model

In [17]:
%%time
# Select X and y features
X = df.copy()
y = df.target

# Lists to store predictions and losses
month_predictions = []
month_losses = []

# Stratified Validation
folds = StratifiedKFold(n_splits = 10, shuffle = True, random_state=seed)

for i,( train_index, test_index) in enumerate(folds.split(X, y)):
  X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

  # Parameters and configuration
  data_config = DataConfig(
      target=['target'],
      continuous_cols = m_test.columns.tolist(),
      categorical_cols = [],
      normalize_continuous_features = True,
      # continuous_feature_transform = 'box-cox',
      num_workers = 4
  )

  trainer_config = TrainerConfig(
      auto_lr_find=True, 
      batch_size=2048,
      max_epochs=500,
      gpus=-1, 
      load_best = True,
      early_stopping_min_delta = 0.01,
      early_stopping_patience = 20
  )
  optimizer_config = OptimizerConfig() 

  model_config = CategoryEmbeddingModelConfig(
      task="classification",
      layers="2048-1024-512", 
      activation="LeakyReLU", 
      use_batch_norm = False,
      dropout = 0.5,
      seed = 5050
  )

  # Instantiate model
  tabular_model = TabularModel(
      data_config=data_config,
      model_config=model_config,
      optimizer_config=optimizer_config,
      trainer_config=trainer_config,
  )

  # Train Model
  tabular_model.fit(train=X_train, validation=X_test)
  result = tabular_model.evaluate(X_test)

  # Print loss per fold
  print(f'\n \033[92m Fold:{i}__{log_loss(X_test.target, tabular_model.predict(X_test).iloc[:,X_test.shape[1]:-1].values)}')

  #Append predictions and losses
  month_losses.append(log_loss(X_test.target, tabular_model.predict(X_test).iloc[:,X_test.shape[1]:-1].values))
  month_predictions.append(tabular_model.predict(tess).iloc[:,tess.shape[1]:-1].values)

# Print CV score
print(f'\n \033[92m Mean Loss: {np.mean(month_losses)}')

Global seed set to 5050
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 650   
2 | backbone               | FeedForwardBackbone | 3.3 M 
3 | output_layer           | Linear              | 4.6 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
13.184    Total estimated model params size (MB)
Global seed set to 5050


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


LR finder stopped early after 92 steps due to diverging loss.
Restored states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 0.0013182567385564075
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 0     
1 | normalizing_batch_norm | BatchNorm1d         | 650   
2 | backbone               | FeedForwardBackbone | 3.3 M 
3 | output_layer           | Linear              | 4.6 K 
4 | loss                   | CrossEntropyLoss    | 0     
---------------------------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
13.184    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Generating Predictions...:   0%|          | 0/5 [00:00<?, ?it/s]


  Fold:9__0.6288274732434651


Generating Predictions...:   0%|          | 0/5 [00:00<?, ?it/s]

Generating Predictions...:   0%|          | 0/18 [00:00<?, ?it/s]


  Mean Loss: 0.5946841001092569
CPU times: user 16min 59s, sys: 10min 13s, total: 27min 12s
Wall time: 1h 11min 8s


In [18]:
blend = np.mean(season_predictions, 0) * 0.6 + np.mean(month_predictions, 0) * 0.4
sub_creator(blend, 'brainiac_pytorch_models')